In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Reading TFIDF Avg w2v vectors(200-D)

In [2]:
tfidf_avg_w2v =pd.read_pickle("\\Users\\AKSHAY\\Desktop\\ML python course\\Amazon Project\\Logistic Regression\\W2V Vectors\\TFIDF Avg W2V Vectors\\tfidf_avg_vec_300")
print(tfidf_avg_w2v.shape)
tfidf_avg_w2v.head()

(122109, 301)


,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,0.000381,0.000057,0.000214,-0.000652,0.000569,0.000155,-0.000022,0.000012,0.000133,-0.000237,...,0.000159,-2.928514e-04,-0.000388,0.000373,0.000125,0.000490,0.000044,0.000365,0.000088,1
1,-0.000259,-0.000003,-0.000142,-0.000090,-0.000162,-0.000175,-0.000015,0.000006,0.000229,0.000259,...,0.000358,1.465296e-04,0.000333,-0.000071,-0.000077,-0.000467,0.000411,0.000016,0.000237,1
2,-0.000163,0.000338,-0.000283,0.000031,-0.000362,-0.000383,0.000013,-0.000628,-0.000422,-0.000324,...,0.000022,-3.309200e-05,-0.000097,0.000375,0.000341,0.000211,-0.000130,0.000439,-0.000269,0
3,-0.000295,0.000153,-0.000261,0.000144,-0.000130,-0.000072,0.000244,-0.000211,0.000115,0.000338,...,0.000165,-8.343711e-05,-0.000220,0.000138,-0.000048,0.000064,-0.000270,0.000172,-0.000278,1
4,-0.000332,-0.000007,0.000646,0.000541,-0.000582,0.000631,-0.000412,-0.000221,0.000392,-0.000698,...,-0.000310,6.762692e-07,-0.000050,-0.000476,0.000319,0.000305,0.000251,0.000152,-0.000083,1


In [4]:
x =tfidf_avg_w2v.iloc[:,:300]
y =tfidf_avg_w2v.iloc[:,300]
print(x.shape)
y.shape

(122109, 300)


(122109,)

## Column Standardization

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
a =StandardScaler()
x =a.fit_transform(x)

In [7]:
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size =0.3)

In [8]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
y_test.shape

(85476, 300)
(36633, 300)
(85476,)


(36633,)

In [9]:
tuned_params = [{'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}]

In [10]:
model = GridSearchCV(LogisticRegression(), tuned_params, scoring = 'accuracy')
model.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [12]:
print(model.best_estimator_)
print(model.score(x_test, y_test))

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.711653427238


In [13]:
y_pred = model.best_estimator_.predict(x_test)

In [14]:
acc = accuracy_score(y_test, y_pred, normalize=True) * float(100)
acc

71.165342723773648

We can clearly see increasing dimenions the model performs better than 100-D,200-D as the chances of data becomes linearly separable in higher dimensions increases.So we get the best hyperplane as dimensions increases due to which model behaves better.

## Confusion Matrix

In [15]:
confusion_matrix(y_test,y_pred).T

array([[11490,  4914],
       [ 5649, 14580]], dtype=int64)

The confusion matrix of 300-D is better than other two

## L1 regularization and sparsity

In [16]:
clf = LogisticRegression(C=1, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_    
print(np.count_nonzero(w))

300


In [17]:
clf = LogisticRegression(C=0.1, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_
print(np.count_nonzero(w))

295


As we can see clearly as c decreases sparsity increases.This time 295 features out of 300  are non zero,means 5 features are removed.

In [18]:
clf = LogisticRegression(C=0.001, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_
print(np.count_nonzero(w))

107


In [20]:
clf = LogisticRegression(C=0.00025, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_
print(np.count_nonzero(w))

2
